**MODELOS DE EMBEDDINGS BASADOS EN WORD2VEC**

*John Atkinson*

Este programa utiliza  crea y utiliza modelos de embeddings de lenguaje basado en mètodos del tipo Word2Vec.

Primero, necesitamos instalar algunos paquetes:

In [1]:
!pip install spacy
!python -m spacy download es_core_news_sm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
2022-09-25 03:39:59.583362: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 12.9 MB 2.7 MB/s 
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')


Montamos nuestro *Drive* donde se encuentra la carpeta CORPUS con documentos separados por tema:

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')
%cd /content/gdrive/MyDrive/CORPUS

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
[Errno 2] No such file or directory: '/content/gdrive/MyDrive/CORPUS'
/content


Debemos importar algunas bibliotecas  y utilitarios:

In [ ]:
import es_core_news_sm
from string import punctuation
from gensim.models import Word2Vec, KeyedVectors
import numpy as np
import pandas as pd
import regex
from sklearn.model_selection import train_test_split
from scipy.spatial.distance import cosine
from sklearn.metrics.pairwise import cosine_similarity
import os
from spacy.lang.es.stop_words import STOP_WORDS
from string import punctuation
import matplotlib.pyplot as plt

Primero, definimos la función **EntrenarModelo(oraciones,NombreModelo)**, que permite entrenar un modelo Word2Vec a partir de un conjunto de oraciones extraída desde un corpus. El modelo generado se graba en la carpeta **NombreModelo**. Asumimos que la ventaja de entrenamiento es 2 ($windows=2$) y que el número de dimensiones o tamaño del vector es 4 ($size=4$).

Luego, definimos una función **CargarModelo(NombreModelo)**, que nos permitirá cargar un modelo cuando sea necesario.

Note que no necesariamente debemos entrenar el modelo nosotros mismos. Podríamos cargar un modelo de embeddings que ya ha sido entrenado por alguien más.

In [ ]:
def EntrenarModelo(oraciones,NombreModelo):
    model = Word2Vec(oraciones, size=4, window=2, min_count=1)
    model.save(NombreModelo)
    
def CargarModelo(NombreModelo):
   modelo = Word2Vec.load(NombreModelo)
   vocabulario = [term for term in modelo.wv.vocab]  
   return(modelo,vocabulario)

Además, necesitaremos una función **ObtenerEmbeddingOracion(modelo, oracion)**, que nos permitirá obtener el embedding (vector) de una oración desde un modelo entrenado. El embedding de una oración es simplemente el vector promedio de cada una de las palabras de la oración: 

In [ ]:
def ObtenerEmbeddingOracion(modelo, oracion):
   Lista_vectores = []
   for w in Tokenizar(oracion):
       # Verificar que la palabra w exista en el modelo
       try:
           modelo.wv[w]
       except KeyError:
           continue
       # Obtener vector de la palabra
       vec = modelo.wv[w]
       Lista_vectores.append(vec)
   embedding_palabras = np.array(Lista_vectores)
   if (len(embedding_palabras) > 0):
        embedding_oracion = embedding_palabras.mean(axis=0)
   else:
        embedding_oracion = np.zeros(modelo.vector_size)
   return(embedding_oracion) 

Ahora, utilizamos algunas funciones de preprocesamiento:

In [ ]:
def PreProcesarOraciones(textos):
    texto_limpio = []
    for texto in textos:  
        if len(texto)!=0:
          texto = regex.sub(' +', ' ', texto)
          tokens = Tokenizar(texto)
          texto_limpio.append(tokens)
    return(texto_limpio)

def Tokenizar(oracion):
    doc = nlp(oracion)
    tokens = [palabra.text for palabra in doc]
    return(tokens)

def CrearCorpus(path):
  directorio = os.listdir(path)
  corpus = []
  doc_id = []  
  for filename  in directorio:
     texto = open(path+filename,'r',encoding="latin-1").read()
     corpus.append(texto)
     doc_id.append(filename)
  return(corpus,doc_id)

En caso de ser necesario definimos una función que permite convertir una lista a un diccionario, de modo de poder acceder por clave (y no por índice):

In [ ]:
def CrearDiccionario(lista,claves):
   dicc = {}
   for  v in range(0,len(claves)):
      dicc[claves[v]] = lista[v]
   return(dicc)

Ahora, ejecutamos nuestro programa principal con algunas incializaciones:

In [ ]:
PATH = "deportes/"
nlp          = es_core_news_sm.load()
corpus,docID = CrearCorpus(PATH)
oraciones    =  PreProcesarOraciones(corpus)
CorpusConClave  = CrearDiccionario(corpus,docID)

Entrenamos el modelo en base a las oraciones generadas previamente:

In [ ]:
EntrenarModelo(oraciones,'mi_word2vec')

Luego, cargamos el modelo entrenado:

In [ ]:
modelo, vocabulario = CargarModelo('mi_word2vec')

Podemos, obtener el embedding de alguna palabra:

In [ ]:
print(modelo.wv['jugador'])

Una vez que tenemos nuestro modelo cargado, podemos realizar diferentes tareas sobre los vectores de palabras u oraciones.

Por ejemplo, podemos determinar la *cercanía* entre dos documentos del corpus. Para ello:

1.   Tomamos el texto de cada documento.
2.   Obtenemos sus respectivos vectores (embeddings).
3.   Calculamos la distancia **coseno**.

In [ ]:
doc1 = CorpusConClave['d15.txt']
doc2 = CorpusConClave['d21.txt']
vec1 = ObtenerEmbeddingOracion(modelo, doc1)
vec2 = ObtenerEmbeddingOracion(modelo, doc2)

similitud = 1-cosine(vec1,vec2)
print(similitud)